In [1]:
import matplotlib.pyplot as plt
import numpy as np
import  transformers
import transformers.models
from transformers import AutoTokenizer
from datasets import load_dataset
config = transformers.models.bert.BertConfig()
model = transformers.models.bert.BertForMaskedLM(config)

In [18]:
# model.from_pretrained("bert-base-multilingual-cased")
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-multilingual-cased")

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from glob import glob
import tqdm

In [23]:
langset= "jaen_eval_past100"
paths = glob(f"../data/awesome_eval_data/{langset}.*")
# print(paths)
df = pd.DataFrame({path.split(f'/')[-1].replace(f"{langset}.", ""): open(path, "r", encoding='utf-8').read().split('\n')[:-1] for path in paths})
# datum = df.iloc[130] # ["nofinetune.out"]
def prepare_raw_datum(raw_gold, raw_hypothesis, raw_srt_tgt):
    src, tgt = [d.strip().split(" ") for d in raw_srt_tgt.split(" ||| ")]
    m = len(src)
    l = len(tgt)
    gold_mat = np.zeros((m, l)) # assume gold is 1 indexed, so subtract by 1.
    sub = 1
    if "jaen" in langset and "awesome_test_examples" not in paths[0]:
        sub = 0
    possible_gold_mat = [tuple(int(i)-sub for i in out.replace('-', 'p').split('p')) for out in raw_gold.strip().split(' ')]
    sure_gold_mat = [tuple(int(i)-sub for i in out.split('-')) for out in raw_gold.strip().split(' ') if "-" in out]
    gold_mat[tuple(zip(*possible_gold_mat))] = 0.5
    gold_mat[tuple(zip(*sure_gold_mat))] = 1

    out_mat = np.zeros((m, l))
    sure_hypo_mat = [tuple(int(i)-sub for i in out.split('-')) for out in raw_hypothesis.split(' ') if "-" in out]
    out_mat[tuple(zip(*sure_hypo_mat))] = 1

    return gold_mat, out_mat, src, tgt
def get_prec_recall_aer(sure, possible, hypothesis, srcs, tgts, return_impact=False):
    total_sure = 0
    total_guessed = 0
    # recall is the ones you can get right out of the ones there are to get right.
    # this is the A intersect sure / sure
    total_recall_numerator = 0

    # precision is the ones you get right out of the ones you guessed.
    # this is A interect possible / A
    total_precision_numerator = 0
    coverages = []

    for s, p, h in zip(sure, possible, hypothesis):
        total_sure += len(s)
        total_guessed += len(h)
        total_recall_numerator += len(set(s).intersection(h))
        total_precision_numerator += len(set(h).intersection(p))
    precision = total_precision_numerator / total_guessed
    recall = total_recall_numerator / total_sure
    aer = 1 - (total_recall_numerator + total_precision_numerator) / (total_sure + total_guessed)

    num_sure = 0
    guesses_made = 0
    guesses_made_in_possible = 0
    guesses_made_in_sure = 0

    num_words_total = 0
    num_words_covered = 0
    coverages = []
    for gold_sure_word_alignment, gold_possible_word_alignment, word_level_alignment, src, tgt in zip(sure, possible, hypothesis, srcs, tgts):
    
        num_sure += len(gold_sure_word_alignment)
        guesses_made += len(word_level_alignment)
        guesses_made_in_sure += len(set(word_level_alignment).intersection(gold_sure_word_alignment))
        guesses_made_in_possible += len(set(word_level_alignment).intersection(gold_possible_word_alignment))
        num_words_total += len(src) + len(tgt) # this doesn't account for the fact we only take the first 510 tokens to align.
        num_words_covered += len(set(s for s,t in word_level_alignment)) + len(set(t for s,t in word_level_alignment))
        coverages.append((len(set(s for s,t in word_level_alignment)) + len(set(t for s,t in word_level_alignment))) / (len(src) + len(tgt)))
    precision2 = guesses_made_in_possible / guesses_made
    recall2 = guesses_made_in_sure / num_sure
    aer2 = 1 - ((guesses_made_in_possible + guesses_made_in_sure) / (guesses_made + num_sure))
    coverage = sum(coverages) / len(coverages)
    coverage2 = num_words_covered / num_words_total
    assert (precision == precision2)
    assert (recall == recall2)
    assert (aer == aer2)

    if return_impact:
        impact = (total_sure + total_guessed) - (total_recall_numerator + total_precision_numerator)
        return precision2, recall2, aer2, coverage, coverage2, impact
    else:
        return precision2, recall2, aer2, coverage, coverage2

# display_alignment(*prepare_raw_datum(datum["gold"], datum['nofinetune.out'], datum['src-tgt']))

for out_format in ["gold"]: # , "fast-grow-diagonal"
    sure = []
    possible = []
    hypothesis = []
    srcs = []
    tgts = []
    for i in range(len(df)):
        datum = df.iloc[i]
        gold_mat, out_mat, src, tgt = prepare_raw_datum(datum["gold"], datum["gold"], datum['src-tgt'])
        
        s = list(zip(*np.where(gold_mat==1)))
        p = list(zip(*np.where(gold_mat>=0.5)))
        h = list(zip(*np.where(out_mat==1)))
        df.loc[i,out_format + "precision"], df.loc[i,out_format + "recall"], df.loc[i,out_format + "aer"], df.loc[i,out_format + "coverage"], df.loc[i,out_format + "coverage2"], df.loc[i, out_format + "impact_on_aer"] = get_prec_recall_aer([s], [p], [h], [src], [tgt], return_impact=True) # type: ignore
        
        sure.append(s)
        possible.append(p)
        hypothesis.append(h)
        srcs.append(src)
        tgts.append(tgt)
    print(out_format)
    print(get_prec_recall_aer(sure, possible, hypothesis, srcs, tgts))
    # print(df.sort_values(out_format + 'impact_on_aer').index.tolist()[::-1])

gold
(1.0, 1.0, 0.0, 0.8136860244632429, 0.7896173224568138)


In [19]:
# jaen_eval_past100: 0.7896173224568138

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw